## Processing the Seoul living population dataset

In this notebook, I processed the living population dataset in the seoul open data portal for the data dashboard (index.html). As a result of the notebook, a csv file which contained the average of living populations of each administrative neighborhood (행정동) was created.

In [1]:
import numpy as np
import pandas as pd

In [2]:
import os

In [3]:
dataset_address = os.listdir('data by month')

In [4]:
dataset_dict = {}
for d in dataset_address:
    month = d.replace('.csv','')
    target_ = 'data by month/'+d
    dataset_dict[month] = pd.read_csv(target_)

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def processing_(month, month_dataset):
    df = month_dataset.copy() 
    df = df.groupby(['행정동코드', '시간대구분'],as_index=False).mean()
    male = df.columns.tolist()[4:18]
    female = df.columns.tolist()[18:]
    
    df['male'] = df[male].sum(axis=1)
    df['female'] = df[female].sum(axis=1)
    
    df = df.rename(columns={'행정동코드':'adm_code',
                            '시간대구분':'time',
                            '총생활인구수':'total'})
    
    df = df[['adm_code','time','total','male','female']]
    df = df.astype('int32')
    df = df.pivot(index='adm_code', columns='time',values=['total','male','female'])
    columns = []
    
    for p in df.columns.levels[0]:
        for i in df.columns.levels[1]:
            columns.append(p+'_'+month+'_'+str(i))
            
    df.columns = columns
    df['total_'+month+'_avg']=df[df.columns[:24]].mean(axis=1)
    df['male_'+month+'_avg']=df[df.columns[24:48]].mean(axis=1)
    df['female_'+month+'_avg']=df[df.columns[48:72]].mean(axis=1)
    return df

In [6]:
for d in dataset_dict.keys():
    print(d)
    dataset_dict[d] = processing_(d,dataset_dict[d]) 

Apr
Aug
Dec
Feb
Jan
Jul
Jun
Mar
May
Nov
Oct
Sept


In [7]:
living_pop = pd.concat([dataset_dict[d] for d in dataset_dict.keys()],axis=1)

In [8]:
living_pop.head()

,total_Apr_0,total_Apr_1,total_Apr_2,total_Apr_3,total_Apr_4,total_Apr_5,total_Apr_6,total_Apr_7,total_Apr_8,total_Apr_9,...,female_Sept_17,female_Sept_18,female_Sept_19,female_Sept_20,female_Sept_21,female_Sept_22,female_Sept_23,total_Sept_avg,male_Sept_avg,female_Sept_avg
adm_code,,,,,,,,,,,,,,,,,,,,,
11110515,15268,15310,15303,15300,15474,15579,16285,18373,20585,21407,...,9239,8466,8302,8010,7594,7374,7653,16705.750000,7778.875000,8926.541667
11110530,21557,21028,20605,21748,22767,22054,24529,31030,42664,49153,...,22095,19834,17362,15250,12936,10708,8673,30634.083333,15099.166667,15534.333333
11110540,4643,4582,4628,4632,4658,4773,5161,6451,7760,9197,...,5082,4232,3679,3113,2576,2302,2412,7322.625000,3685.041667,3637.125000
11110550,14292,13985,13915,13874,13242,13302,13524,13965,14343,14536,...,7429,7364,7494,7643,7743,7877,7225,13677.541667,6080.000000,7597.041667
11110560,21776,22122,22421,22484,22887,23129,23092,22077,20993,20142,...,10074,9945,10108,10417,10819,11205,12665,20909.375000,9270.125000,11638.791667


In [9]:
time_list = [t for t in range(24)]
time_list.append('avg')
test = {}
for i in time_list:
    in_ = str(i)
    not_in = [str(s) for s in range(24) if s != i]
    test[in_] = [k for k in living_pop.columns if in_ in k]
    if in_.isdigit():
        if i<10:
            for n in not_in:
                test[in_]  = [k for k in test[in_] if n not in k]

In [10]:
for t in test.keys():
    living_pop['total_'+'avg_'+t] =living_pop[[t for t in test[t] if 'total' in t]].mean(axis=1)
    living_pop['male_'+'avg_'+t] =living_pop[[t for t in test[t] if 'male' in t]].mean(axis=1)
    living_pop['female_'+'avg_'+t] =living_pop[[t for t in test[t] if 'female' in t]].mean(axis=1)

In [11]:
living_pop['gu'] = living_pop.index // 1000

In [12]:
total_columns = [c for c in living_pop.columns if 'total' in c]
living_pop['max'] = living_pop[total_columns].max(axis=1)
living_pop['min'] = living_pop[total_columns].min(axis=1)

In [13]:
living_pop['min'] 

adm_code
11110515    13533.0
11110530    14441.0
11110540     4392.0
11110550    11187.0
11110560    16358.0
             ...   
11740650    23545.0
11740660    25451.0
11740685    44958.0
11740690     4333.0
11740700    25698.0
Name: min, Length: 424, dtype: float64

In [14]:
coord = pd.read_csv('dong_coord.csv',index_col=0)
coord = coord[['adm_cd2','lat','lng']]
coord.adm_cd2 = coord.adm_cd2//100
living_pop = living_pop.merge(coord,left_index=True, right_on='adm_cd2')
living_pop = living_pop.set_index('adm_cd2')

living_pop.to_csv('living_pop_neighborhood.csv')

In [25]:
living_pop_total = living_pop.sum(axis=0)

In [26]:
living_pop_total = living_pop_total.drop(['gu','lat','lng','max','min'])

In [27]:
living_pop_total = pd.DataFrame(living_pop_total).rename(columns={0:'pop'})

In [29]:
living_pop_total = living_pop_total.reset_index().rename(columns={'index':'month'})

In [35]:
living_pop_total['time'] = living_pop_total['month'].str.split('_').apply(lambda x: x[-1])
living_pop_total['type'] = living_pop_total['month'].str.split('_').apply(lambda x: x[0])
living_pop_total['month'] = living_pop_total['month'].str.split('_').apply(lambda x: x[1])

In [37]:
living_pop_total.to_csv('living_population_total.csv',index=False)